In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = 1200
DEFAULT_END = "_end"
DEFAULT_PASS =" "
DEFAULT_UNK =''

разбивка текста и создание словаря

In [19]:
from Tokenizer import Tokenizer
text = open("text.txt",encoding="utf-8").read().lower()
words = text.split()
questions = text.splitlines()[::2]
answers = text.splitlines()[1::2]
tokenizer = Tokenizer(VOCAB_SIZE)
tokenizer.fit([text],DEFAULT_END,DEFAULT_UNK)

создание модели 
предложение -> hidden
последние слово + hidden -> слово(1)...слово(n)

In [20]:
class RnnTextGen(nn.Module):

    def __init__(self,voc_size,inp_size,hid_size,n_layers,dropout=0.2) -> None:
        super(RnnTextGen,self).__init__()
        self.voc_size = voc_size
        self.n_layers = n_layers
        self.hidden_size=hid_size
        self.Encoder = nn.Embedding(voc_size,inp_size)
        self.lstm = nn.LSTM(inp_size,hid_size,n_layers)
        self.dropout = nn.Dropout(dropout)
        self.l1 = nn.Linear(hid_size,voc_size)
        
    def forward(self,x,hidden=None):
        x = self.Encoder(x)
        x,hidden = self.lstm(x)
        x = self.dropout(x)
        x = self.l1(x)
        return x,hidden
    
    def init_hidden(self,batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [21]:
model=RnnTextGen(VOCAB_SIZE,1000,500,2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)


In [22]:
def evaluate(model:RnnTextGen,text:str,prediction_lim:int=15):
    text_idx = torch.LongTensor(list(tokenizer.tokenize(text))).to(device)
    hidden = model.init_hidden()
    inp = text_idx
    predicted_text=""
    for i in range(prediction_lim):
        next_w , hidden = model(inp.view(-1,1).to(device),hidden)
        inp = torch.cat([inp,next_w[-1].argmax().view(-1)])
        word = tokenizer.rw_tokens[int(next_w[-1].argmax())]
        if next_w[-1].argmax() == torch.LongTensor([0]).to(device):
            break
        predicted_text +=word
    return predicted_text

In [23]:
def get_batch(questions:list,answers:list):
    for question,answer in zip(questions,answers):
        question_idx = list(tokenizer.tokenize(question))
        target = list(tokenizer.tokenize(answer))+[0]
        test = question_idx+target[:-1]

        target =torch.LongTensor(target).to(device)
        test = torch.LongTensor(test).to(device)
        yield target,test

In [24]:
def train(epoches:int,model:RnnTextGen,batch_size:int)->None:
    """epoches - number of epoches through all dataset
    model - model required to teach
    batch_size - n/a"""
    loss_avg =[]
    for epoch in range(epoches):
        for target,train in get_batch(questions,answers):
            model.train()

            hidden = model.init_hidden(batch_size)

            output,hidden = model(train,hidden)
            target_len = len(target)
            loss = criterion(output[-target_len:],target)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_avg.append(loss.item())
            if len(loss_avg) >= 50:
                mean_loss = np.mean(loss_avg)
                print(f'Loss: {mean_loss}')
                scheduler.step(mean_loss)
                model.eval()
                question = random.choice(questions)
                answer = evaluate(model,question)
                print(f"Question: {question} \n Answer: {answer}")
                loss_avg = []

обучение модели

In [25]:
train(10,model,1)

Loss: 6.5692675685882564
Question: какую роль играет учитель в образовании? 
 Answer: я ду, чт чт.
Loss: 5.60169397354126
Question: существует ли свободная воля? 
 Answer: мой любимый д. а у тебя?мой любимый 
Loss: 5.607483897209168
Question: но как мы можем узнать истину? 
 Answer: 
Loss: 4.852296171188354
Question: а скоро кончишь? 
 Answer: это может быть сложно, но я думаю, что чело
Loss: 4.128365134000778
Question: куда ты собираешься? 
 Answer: мой любимый й уддд. а у тебя?
Loss: 4.156370911598206
Question: ты лох  
 Answer: вый
Loss: 3.6160613006353377
Question: какой твой любимый цвет? 
 Answer: 
Loss: 2.6554657512903215
Question: софия как ты считаешь как можно получить практический опыт? 
 Answer: я ел оет - гр. а у тебя?
Loss: 2.9044772958755494
Question: но как мы можем определить, что правильно, а что нет? 
 Answer: ничего
Loss: 2.4053306111693384
Question: софия как ты считаешь как можно получить практический опыт? 
 Answer: это может быть сложно, но я думаю, что наш
Loss

In [26]:
quest = input()
evaluate(model,quest,35)

'привет '

In [27]:
print(tokenizer.rw_tokens)

{0: '_end', 1: '', 2: 'т', 3: 'р', 4: 'з', 5: 'м', 6: '\n', 7: ',', 8: '-', 9: 'ш', 10: 'х', 11: 'в', 12: 'ч', 13: 'а', 14: '"', 15: 'д', 16: 'е', 17: 'ю', 18: 'щ', 19: ':', 20: '.', 21: 'н', 22: 'й', 23: '0', 24: 'ж', 25: 'э', 26: 'с', 27: 'и', 28: 'ь', 29: 'ф', 30: 'у', 31: 'л', 32: '*', 33: 'ц', 34: 'к', 35: 'п', 36: ' ', 37: '2', 38: 'г', 39: 'я', 40: 'б', 41: 'о', 42: 'ы', 43: '?', 44: 'е ', 45: 'и ', 46: 'о ', 47: 'ь ', 48: 'ть ', 49: 'й ', 50: 'ак', 51: 'как', 52: ' н', 53: ' може', 54: ' п', 55: 'я ', 56: ' и', 57: ' м', 58: 'мо', 59: ' на', 60: 'ка', 61: 'ра', 62: 'т ', 63: ' мо', 64: 'на', 65: 'ст', 66: ' с', 67: ' к', 68: 'ть', 69: ' пр', 70: 'пр', 71: 'то', 72: ' мож', 73: 'може', 74: 'как ', 75: 'во', 76: ' и ', 77: ' т', 78: 'а ', 79: ' в', 80: 'но', 81: ' как', 82: 'ни', 83: 'ет', 84: 'то ', 85: ' как ', 86: 'ел', 87: ' ка', 88: 'ой ', 89: 'можем', 90: 'ожем ', 91: 'ет ', 92: 'мож', 93: 'ы ', 94: ' о', 95: 'мы мо', 96: 'ы мож', 97: 'ак ', 98: 'ве', 99: 'оже', 100: 'ен', 

In [28]:
torch.save(model,"data.pkl")

In [29]:
model = torch.load("data.pkl").to(device)